In [ ]:
# %pip install mlxtend

In [8]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
import os

# ==========================================
# CẤU HÌNH (Sửa tại đây nếu cần)
# ==========================================
# Số lượng đơn hàng lấy mẫu. 
# Nếu máy RAM 8GB -> Để 50000. RAM 16GB -> Để 100000.
SAMPLE_SIZE = 100000 

# Độ phổ biến tối thiểu (1%). Sản phẩm phải xuất hiện trong 1% giỏ hàng mới tính.
MIN_SUPPORT = 0.01 

print("🚀 BẮT ĐẦU QUY TRÌNH KHAI PHÁ DỮ LIỆU INSTACART...")
print(f"--> Cấu hình: Sample={SAMPLE_SIZE}, Min_Support={MIN_SUPPORT}")

# ==========================================
# 1. LOAD DỮ LIỆU (Tự động tìm đường dẫn)
# ==========================================
try:
    # Định nghĩa các đường dẫn có thể xảy ra
    possible_paths = [
        ('data/order_products__train.csv', 'data/products.csv'),       # Chạy từ root
        ('../data/order_products__train.csv', '../data/products.csv')  # Chạy từ subfolder
    ]
    
    path_orders = None
    path_products = None

    for p_orders, p_products in possible_paths:
        if os.path.exists(p_orders) and os.path.exists(p_products):
            path_orders = p_orders
            path_products = p_products
            break
    
    if path_orders is None:
        raise FileNotFoundError("❌ Không tìm thấy thư mục 'data'. Hãy kiểm tra lại cấu trúc dự án!")

    print(f"⏳ Đang đọc dữ liệu từ: {path_orders}...")
    
    # Chỉ load các cột cần thiết để tiết kiệm RAM
    df_orders = pd.read_csv(path_orders, usecols=['order_id', 'product_id'])
    df_products = pd.read_csv(path_products, usecols=['product_id', 'product_name'])
    
    # LẤY MẪU DỮ LIỆU (Bước quan trọng để không tràn RAM)
    df_orders = df_orders.head(SAMPLE_SIZE)

    # Merge bảng để lấy tên sản phẩm
    df = pd.merge(df_orders, df_products, on='product_id')
    print(f"✅ Load xong! Tổng số dòng dữ liệu đang xử lý: {len(df)}")

except Exception as e:
    print(f"❌ LỖI LOAD DATA: {e}")
    raise

# ==========================================
# 2. BIẾN ĐỔI DỮ LIỆU (ONE-HOT ENCODING)
# ==========================================
print("⏳ Đang chuyển đổi sang ma trận Giỏ hàng (Bước này tốn RAM nhất)...")

try:
    # Tạo bảng Pivot: Hàng = OrderID, Cột = Tên Sản Phẩm
    basket = (df.groupby(['order_id', 'product_name'])['product_name']
              .count().unstack().reset_index().fillna(0)
              .set_index('order_id'))

    # Tối ưu hóa: Chuyển đổi số lượng thành 0 và 1 (Boolean)
    # Cách này nhanh hơn applymap gấp nhiều lần
    basket_sets = (basket > 0).astype(int)

    # Xóa cột phí ship nếu có
    if 'POSTAGE' in basket_sets.columns:
        basket_sets.drop('POSTAGE', inplace=True, axis=1)

    print(f"✅ Đã tạo ma trận thành công. Kích thước: {basket_sets.shape}")

except Exception as e:
    print(f"❌ LỖI XỬ LÝ MA TRẬN: {e}")
    print("GỢI Ý: Hãy giảm SAMPLE_SIZE xuống thấp hơn (ví dụ 20000).")
    raise

# ==========================================
# 3. CHẠY THUẬT TOÁN FP-GROWTH
# ==========================================
print(f"⏳ Đang chạy thuật toán FP-Growth...")
frequent_itemsets = fpgrowth(basket_sets, min_support=MIN_SUPPORT, use_colnames=True)

if frequent_itemsets.empty:
    print("⚠️ CẢNH BÁO: Không tìm thấy tập phổ biến nào! Hãy giảm MIN_SUPPORT xuống (ví dụ 0.005).")
else:
    print(f"✅ Tìm thấy {len(frequent_itemsets)} cụm sản phẩm phổ biến.")

    # ==========================================
    # 4. TẠO LUẬT KẾT HỢP (ASSOCIATION RULES)
    # ==========================================
    print("⏳ Đang sinh luật kết hợp và tính toán chỉ số Lift...")
    
    # min_threshold=1: Chỉ lấy các luật có tác động tích cực (Lift >= 1)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    # Sắp xếp: Luật nào mạnh nhất (Lift cao) và chắc chắn nhất (Confidence cao) lên đầu
    rules = rules.sort_values(['lift', 'confidence'], ascending=[False, False])

    # ==========================================
    # 5. LÀM SẠCH VÀ LƯU KẾT QUẢ
    # ==========================================
    print("⏳ Đang format lại dữ liệu để lưu...")

    # Chuyển đổi frozenset thành chuỗi text (VD: {'Tao', 'Chuoi'} -> "Tao, Chuoi")
    # Điều này giúp lưu vào Database dễ dàng và giữ được luật Combo
    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

    # Tạo thư mục output nếu chưa có
    if not os.path.exists('output'):
        os.makedirs('output')
    
    output_path = 'output/ket_qua_luat_final.csv'
    
    # Lưu file CSV
    # Chỉ lấy các cột quan trọng
    cols_to_save = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
    rules[cols_to_save].to_csv(output_path, index=False)

    print("\n" + "="*50)
    print("🎉 HOÀN THÀNH XUẤT SẮC!")
    print("="*50)
    print("Top 5 luật tìm được:")
    print(rules[cols_to_save].head(5))
    print(f"\n📂 File kết quả đã lưu tại: {output_path}")
    print("👉 Hãy gửi file này cho bạn Backend để Import vào Database.")

🚀 BẮT ĐẦU QUY TRÌNH KHAI PHÁ DỮ LIỆU INSTACART...
--> Cấu hình: Sample=100000, Min_Support=0.01
⏳ Đang đọc dữ liệu từ: ../data/order_products__train.csv...
✅ Load xong! Tổng số dòng dữ liệu đang xử lý: 100000
⏳ Đang chuyển đổi sang ma trận Giỏ hàng (Bước này tốn RAM nhất)...
✅ Đã tạo ma trận thành công. Kích thước: (9477, 16738)
⏳ Đang chạy thuật toán FP-Growth...


C:\Users\nguye\AppData\Roaming\Python\Python312\site-packages\mlxtend\frequent_patterns\fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


✅ Tìm thấy 128 cụm sản phẩm phổ biến.
⏳ Đang sinh luật kết hợp và tính toán chỉ số Lift...
⏳ Đang format lại dữ liệu để lưu...

🎉 HOÀN THÀNH XUẤT SẮC!
Top 5 luật tìm được:
             antecedents           consequents   support  confidence      lift
30           Large Lemon                 Limes  0.012135    0.193603  4.410511
31                 Limes           Large Lemon  0.012135    0.276442  4.410511
11   Organic Blueberries  Organic Strawberries  0.010130    0.285714  3.489323
10  Organic Strawberries   Organic Blueberries  0.010130    0.123711  3.489323
7    Organic Raspberries  Organic Strawberries  0.011185    0.273196  3.336440

📂 File kết quả đã lưu tại: output/ket_qua_luat_final.csv
👉 Hãy gửi file này cho bạn Backend để Import vào Database.
